# Necessary Tools and Services

In [32]:
# ! pip install pandas
# ! pip install matplotlib
# ! pip install numpy
# ! pip install seaborn
# ! pip install unzip
# ! pip install gensim
# ! pip install nltk
# ! pip install wordcloud
# ! pip install spacy
# ! pip install spacy_download
# ! pip install pyLDAvis
# ! pip install PyStemmer

# ! python3 -m spacy download en

In [1]:
## Importing PD and Others
import re
import numpy as np
import pandas as pd
from pprint import pprint

## Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

## NLTK
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk.stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['#', '`', '"', '@'])

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning)

import matplotlib.pyplot as plt
%matplotlib inline


import spacy
spacy.cli.download('en_core_web_sm')



## Visualization
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uji657/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /home/uji657/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /home/uji657/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.7.1/en_core_web_sm-3.7.1-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [22]:
def FindingPostsWithNegativeSentiments():
    df = pd.read_csv('../dataset/SO_Workflow_Data.csv')
    my_value = -1
    data = df.loc[df["RatingsGPTFineTuned"] == my_value]

    df = data[data['Body'].str.contains('workbench', case=False)] 
    print(len(df))
    # df.to_csv('../dataset/workbench_posts.csv', index=False)

FindingPostsWithNegativeSentiments()

21


# Import data and Preprocess

In [2]:
df = pd.read_csv('../dataset/workbench_posts.csv')
# my_value = -1
# new_df = df.loc[df["RatingsGPTFineTuned"] == my_value]
df["merged"] = df[["Title","Body"]].apply("-".join, axis=1)
#new_df.head()
# new_df.to_csv('Dataset/ConcatenatedDatasetSO.csv')

data = df.merged.values.tolist()
print(len(df))
#data

20


In [3]:
# Remove Emails
data = [re.sub('<[^<>]*>', '', sent) for sent in data]
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]
# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:2])

['Application "org.eclipse.e4.ui.workbench.swt.E4Application" could not be '
 'found in the registry-I have just moved from 4.2 to 4.3 (kepler). I was '
 'working on Eclipse RCP(OSGi framework) which was working fine with 4.2, but '
 'once i moved to Kepler- started getting below mentioned error while '
 'launching. Any kind of help is appreciated. java.lang.RuntimeException: '
 'Application "org.eclipse.e4.ui.workbench.swt.E4Application" could not be '
 'found in the registry. The applications available are: '
 'org.eclipse.equinox.app.error. at '
 'org.eclipse.equinox.internal.app.EclipseAppContainer.startDefaultApp(EclipseAppContainer.java:248) '
 'at '
 'org.eclipse.equinox.internal.app.MainApplicationLauncher.run(MainApplicationLauncher.java:29) '
 'at '
 'org.eclipse.core.runtime.internal.adaptor.EclipseAppLauncher.runApplication(EclipseAppLauncher.java:110) '
 'at '
 'org.eclipse.core.runtime.internal.adaptor.EclipseAppLauncher.start(EclipseAppLauncher.java:79) '
 'at '
 'org.ec

# Tokenize words and Clean-up text

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['application', 'org', 'eclipse', 'ui', 'workbench', 'swt', 'application', 'could', 'not', 'be', 'found', 'in', 'the', 'registry', 'have', 'just', 'moved', 'from', 'to', 'kepler', 'was', 'working', 'on', 'eclipse', 'rcp', 'osgi', 'framework', 'which', 'was', 'working', 'fine', 'with', 'but', 'once', 'moved', 'to', 'kepler', 'started', 'getting', 'below', 'mentioned', 'error', 'while', 'launching', 'any', 'kind', 'of', 'help', 'is', 'appreciated', 'java', 'lang', 'application', 'org', 'eclipse', 'ui', 'workbench', 'swt', 'application', 'could', 'not', 'be', 'found', 'in', 'the', 'registry', 'the', 'applications', 'available', 'are', 'org', 'eclipse', 'equinox', 'app', 'error', 'at', 'org', 'eclipse', 'equinox', 'internal', 'app', 'startdefaultapp', 'java', 'at', 'org', 'eclipse', 'equinox', 'internal', 'app', 'run', 'java', 'at', 'org', 'eclipse', 'core', 'runtime', 'internal', 'adaptor', 'runapplication', 'java', 'at', 'org', 'eclipse', 'core', 'runtime', 'internal', 'adaptor', 'start

In [5]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
#print(trigram_mod[bigram_mod[data_words[0]]])

# Remove Stopwords, Make Bigrams and Lemmatize

In [6]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [7]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['application', 'org', 'eclipse', 'workbench', 'application', 'find', 'registry', 'move', 'kepler', 'working', 'eclipse', 'rcp', 'osgi_framework', 'work', 'fine', 'moved', 'kepler', 'start', 'get', 'mention', 'error', 'launch', 'kind', 'help', 'appreciate', 'application', 'eclipse', 'workbench', 'application', 'find', 'registry', 'application', 'available', 'org', 'eclipse', 'error', 'org', 'eclipse', 'startdefaultapp', 'run', 'eclipse', 'runtime', 'internal', 'adaptor_runapplication', 'runtime', 'internal', 'adaptor', 'start', 'eclipse', 'runtime', 'adaptor', 'eclipsestarter', 'run', 'eclipsestarter', 'runtime', 'adaptor', 'eclipsestarter', 'run', 'eclipsestarter', 'sun_reflect', 'reflect', 'invoke', 'method', 'main', 'invokeframework', 'main', 'main', 'basicrun', 'main', 'main', 'run', 'main', 'launcher', 'main', 'main', 'main', 'java']]


# Create the Dictionary and Corpus needed for Topic Modeling

In [8]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 3), (1, 1), (2, 5), (3, 1), (4, 1), (5, 1), (6, 7), (7, 4), (8, 2), (9, 2), (10, 1), (11, 1), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 2), (18, 1), (19, 1), (20, 1), (21, 9), (22, 1), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 1), (29, 1), (30, 2), (31, 4), (32, 4), (33, 2), (34, 1), (35, 1), (36, 1), (37, 2), (38, 1)]]


In [9]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('adaptor', 3),
  ('adaptor_runapplication', 1),
  ('application', 5),
  ('appreciate', 1),
  ('available', 1),
  ('basicrun', 1),
  ('eclipse', 7),
  ('eclipsestarter', 4),
  ('error', 2),
  ('find', 2),
  ('fine', 1),
  ('get', 1),
  ('help', 1),
  ('internal', 2),
  ('invoke', 1),
  ('invokeframework', 1),
  ('java', 1),
  ('kepler', 2),
  ('kind', 1),
  ('launch', 1),
  ('launcher', 1),
  ('main', 9),
  ('mention', 1),
  ('method', 1),
  ('move', 1),
  ('moved', 1),
  ('org', 3),
  ('osgi_framework', 1),
  ('rcp', 1),
  ('reflect', 1),
  ('registry', 2),
  ('run', 4),
  ('runtime', 4),
  ('start', 2),
  ('startdefaultapp', 1),
  ('sun_reflect', 1),
  ('work', 1),
  ('workbench', 2),
  ('working', 1)]]

In [13]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [14]:
import numpy as np
import tqdm

grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 2
max_topics = 11
step_size = 1
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]

corpus_title = ['75% Corpus', '100% Corpus']

model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('../dataset/lda_tuning_results.csv', index=False)
    pbar.close()

  2%|▏         | 11/540 [00:06<05:04,  1.73it/s]

KeyboardInterrupt: 

In [15]:
results = pd.read_csv('../dataset/lda_tuning_results.csv')
results = results.loc[272:]
results.sort_values(by=['Coherence'], ascending=False)

,Validation_Set,Topics,Alpha,Beta,Coherence
528,100% Corpus,10,0.9099999999999999,0.9099999999999999,0.545106
358,100% Corpus,4,asymmetric,0.9099999999999999,0.530929
523,100% Corpus,10,0.61,0.9099999999999999,0.530743
417,100% Corpus,6,asymmetric,0.61,0.524255
418,100% Corpus,6,asymmetric,0.9099999999999999,0.520599
...,...,...,...,...,...
301,100% Corpus,3,0.01,0.31,0.333287
304,100% Corpus,3,0.01,symmetric,0.333239
326,100% Corpus,3,asymmetric,0.31,0.325645
327,100% Corpus,3,asymmetric,0.61,0.325645


In [10]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=2, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=0.61,
                                           eta=0.91)

# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis


Perplexity:  -5.746086761528718



Coherence Score:  0.42763408926954427


PreparedData(topic_coordinates=              x    y  topics  cluster       Freq
topic                                           
1      0.061537  0.0       1        1  64.515509
0     -0.061537  0.0       2        1  35.484491, topic_info=               Term        Freq       Total Category  logprob  loglift
165           event   20.000000   20.000000  Default  30.0000  30.0000
13         internal  163.000000  163.000000  Default  29.0000  29.0000
307          search   18.000000   18.000000  Default  28.0000  28.0000
304     refactoring    9.000000    9.000000  Default  27.0000  27.0000
628         monetdb    9.000000    9.000000  Default  26.0000  26.0000
..              ...         ...         ...      ...      ...      ...
311    synchronizer    5.906038   11.043663   Topic2  -5.3556   0.4102
26              org    7.670503   57.803494   Topic2  -5.0942  -0.9836
32          runtime    6.645191   36.029945   Topic2  -5.2377  -0.6544
7    eclipsestarter    5.952063   25.185771   Topic2  -5.3478  -0.4065
8             error    5.874959   28.349089   Topic2  -5.3609  -0.5378

[134 rows x 6 columns], token_table=      Topic      Freq           Term
term                                
445       1  0.223359  abstractelist
445       2  0.893434  abstractelist
0         1  0.780256        adaptor
0         2  0.260085        adaptor
2         1  0.478001    application
...     ...       ...            ...
37        2  0.371136      workbench
424       1  0.939078  workbenchpage
424       2  0.044718  workbenchpage
98        1  0.832155         worker
98        2  0.166431         worker

[178 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1])

In [11]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
print(doc_lda)

[(0,
  '0.051*"internal" + 0.029*"workbench" + 0.023*"run" + 0.018*"eclipse" + '
  '0.015*"core" + 0.015*"application" + 0.013*"event" + 0.011*"search" + '
  '0.011*"main" + 0.009*"java"'),
 (1,
  '0.054*"eclipse" + 0.044*"internal" + 0.028*"workbench" + 0.024*"run" + '
  '0.022*"main" + 0.022*"org" + 0.013*"runtime" + 0.010*"error" + '
  '0.009*"workbenchpage" + 0.008*"eclipsestarter"')]


In [12]:
num_topics = 2
topic_distributions = [lda_model[doc] for doc in corpus]

# Extract the dominant topic for each document
df['topic'] = [max(topics, key=lambda x: x[1])[0] for topics in topic_distributions]

# Display the DataFrame with assigned topics
print(df.head())
df.to_csv('../dataset/AssignedTopicWorkbench.csv')
for i in range(num_topics): 
    topic_df = df[df['topic'] == i]
    print('topic: ', i, 'total data: ', len(topic_df))

         Id                                              Title  \
0  18571901  Application "org.eclipse.e4.ui.workbench.swt.E...   
1  18630845  Eclipse Kepler auto exit on loading workbench ...   
2  18175138  Create an Aspect with AJDT in former Java Project   
3  19136334         Eclipse Plugin executes Javafx application   
4  19257900                     My Eclipse Kepler cannot start   

                                                Body  RatingsSentiCR  \
0  I have just moved from 4.2 to 4.3 (kepler). I ...               1   
1  When i start eclipse, on splash loading window...              -1   
2  I decided to handle a problem in a Java projec...              -1   
3  here's my problem:\nI'm developing an Eclipse ...              -1   
4  My Eclipse Kepler is installed on Ubuntu. Afte...              -1   

   RatingsGPT35  RatingsGPTFineTuned  \
0            -1                 -1.0   
1            -1                 -1.0   
2            -1                 -1.0   
3         